In [13]:
#from pyspark import SparkConf, SparkContext
#sc = SparkContext.getOrCreate()

In [14]:
#from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

In [15]:
#def parsePoint(line):
#    values = [float(x) for x in line.replace(',', ' ').split(' ')]
#    return LabeledPoint(values[0], values[1:])

In [47]:
import pixiedust

In [25]:
#df = pixiedust.sampleData("https://raw.githubusercontent.com/markwatsonatx/watson-ml-for-developers/master/data/house-prices.csv")
#display(df)
#type(df)



In [26]:
#df.printSchema()

In [49]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-05799f7f-1507-4fee-abcc-1eb83733287a',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': 'fNfyX-dNo1VW6KIgqewt2APl_FbC66SIGfHqRdtLj6ah'
}

configuration_name = 'os_d0438052eccb4591a48d39cad8f168b8_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema','True')\
  .load(cos.url('customerdata.csv', 'callforcodebocsp-donotdelete-pr-762lyejmogakw0'))
df_data_1.take(5)

[Row(asset=20000, transfreq=20, credit=90, line=10000),
 Row(asset=50000, transfreq=40, credit=95, line=12500),
 Row(asset=100000, transfreq=50, credit=100, line=20000),
 Row(asset=200000, transfreq=80, credit=100, line=20000),
 Row(asset=200000, transfreq=80, credit=90, line=15000)]

In [50]:
df_data_1.printSchema()

root
 |-- asset: integer (nullable = true)
 |-- transfreq: integer (nullable = true)
 |-- credit: integer (nullable = true)
 |-- line: integer (nullable = true)



In [51]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [52]:
assembler = VectorAssembler(inputCols=['asset','transfreq','credit'],outputCol="features")
lr = LinearRegression(labelCol='line', featuresCol='features')
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(df_data_1)

In [53]:
def get_prediction(asset, transfreq, credit):
    request_df = spark.createDataFrame([(asset, transfreq, credit)], ['asset','transfreq','credit'])
    response_df = model.transform(request_df)
    return response_df

In [57]:
response = get_prediction(2400, 40, 0)
response.show()

+-----+---------+------+-----------------+-----------------+
|asset|transfreq|credit|         features|       prediction|
+-----+---------+------+-----------------+-----------------+
| 2400|       40|     0|[2400.0,40.0,0.0]|701.1477502557209|
+-----+---------+------+-----------------+-----------------+



In [58]:
import json
import requests
import urllib3

In [59]:
service_path = 'https://us-south.ml.cloud.ibm.com'
username = '067600af-cb6c-4d46-a370-7ee1cd794c5b'
password = 'd518afdc-99e1-4b88-be81-019940a77254'
instance_id = '6b50131c-e7e8-4e49-b836-93f2300392d9'
model_name = 'Credit Rating Model'
deployment_name = 'Credit Rating Deployment'


In [60]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

In [61]:


pipeline_artifact = MLRepositoryArtifact(pipeline, name="pipeline")
model_artifact = MLRepositoryArtifact(model, training_data=df_data_1, name=model_name, pipeline_artifact=pipeline_artifact)
saved_model = ml_repository_client.models.save(model_artifact)
model_id = saved_model.uid



In [62]:
headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
ml_token = 'Bearer ' + json.loads(response.text).get('token')

In [70]:
deployment_url = service_path + "/v3/wml_instances/" + instance_id + "/published_models/" + model_id + "/deployments/"
deployment_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
deployment_payload = {"type": "online", "name": deployment_name}
deployment_response = requests.post(deployment_url, json=deployment_payload, headers=deployment_header)
deployment_response.text
scoring_url = json.loads(deployment_response.text).get('entity').get('scoring_url')
scoring_url

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/6b50131c-e7e8-4e49-b836-93f2300392d9/published_models/808c8790-b257-454f-a9d3-1a2208b73867/deployments/c2409c20-653e-4937-8ad0-d187d28a078a/online'

In [71]:
def get_prediction_from_watson_ml(asset, transfreq, credit):
    scoring_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
    scoring_payload = {'fields': ['asset','transfreq','credit'], 'values': [[asset, transfreq, credit]]}
    scoring_response = requests.post(scoring_url, json=scoring_payload, headers=scoring_header)
    return scoring_response.text

In [72]:
response = get_prediction_from_watson_ml(2400, 40, 90)
print(response)

{
  "fields": ["asset", "transfreq", "credit", "features", "prediction"],
  "values": [[2400, 40, 90, [2400.0, 40.0, 90.0], 19336.76802948064]]
}
